In [1]:
%matplotlib inline

In [5]:
import matplotlib.pylab as plt
import torch
import numpy as np
import seaborn as sn
sn.set_context("poster")

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils import weight_norm
from torchvision import transforms, datasets

import numpy as np
import scipy
import scipy.stats as st
import scipy.special
import scipy.signal
import scipy.interpolate

import pandas as pd

from os.path import join
import random
from csv import DictWriter

from tqdm.notebook import tqdm
import pickle
# if gpu is to be used
use_cuda = torch.cuda.is_available()
print(use_cuda)
FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
DoubleTensor = torch.cuda.DoubleTensor if use_cuda else torch.DoubleTensor
IntTensor = torch.cuda.IntTensor if use_cuda else torch.IntTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if use_cuda else torch.ByteTensor
ttype = FloatTensor

True


In [11]:
def data_generator(N, seq_length, ttype):
    """
    Args:
        seq_length: Length of the adding problem data
        N: # of data in the set
    """
    X_num = torch.rand([N, 1, seq_length])
    X_mask = torch.zeros([N, 1, seq_length])
    Y = torch.zeros([N, 1])
    for i in range(N):
        positions = np.random.choice(seq_length, size=2, replace=False)
        X_mask[i, 0, positions[0]] = 1
        X_mask[i, 0, positions[1]] = 1
        Y[i,0] = X_num[i, 0, positions[0]] + X_num[i, 0, positions[1]]
    X = torch.cat((X_num, X_mask), dim=1)
    return X.type(ttype), Y.type(ttype)

In [12]:
seq_length = 400

In [13]:
X_train, Y_train = data_generator(50000, seq_length, ttype=ttype)
X_test, Y_test = data_generator(1000, seq_length, ttype=ttype)